<a href="https://colab.research.google.com/github/MudasirIrshad/Agentic_AI_Panaversity_Course/blob/main/PROJECT_DocumentBoot_RAG_FunctionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%pip install -qU pypdf

In [15]:
!pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [16]:
!pip install --upgrade --quiet  langchain-google-genai

In [17]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [18]:
import langchain_google_genai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
)


#PPT (powerpoint) Embedding

In [19]:
# # Install packages
%pip install unstructured
%pip install python-magic
%pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 15.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=a4a19a0e6952a0c61890a9f762ca119379d86ad652c0b8f5e8773e64547206ec
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [22]:
from google.colab import files
ppt = files.upload()

Saving diabetes_in_pakistan.pptx to diabetes_in_pakistan.pptx


In [23]:
from langchain_core.documents import Document


In [24]:
from pptx import Presentation

def extract_text_from_pptx(filepath):
    prs = Presentation(filepath)
    full_text = []
    for slide_number, slide in enumerate(prs.slides):
        slide_text = []
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                slide_text.append(shape.text)
            # Handle tables, charts, etc., as needed
        full_text.append(f"Slide {slide_number + 1}: {' '.join(slide_text)}")
    return "\n".join(full_text)
pptData = extract_text_from_pptx("/content/diabetes_in_pakistan.pptx")

sentence_data = pptData.split("Slide ")
sentence_data.remove("")
doc = []
for i in range(len(sentence_data)):
  doc.append(Document(page_content=sentence_data[i], metadata={"slide_number":i+1}))


In [25]:
%pip install -qU langchain-pinecone langchain-google-genai

In [26]:
from pinecone import Pinecone
from google.colab import userdata

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

pinecone_api_key

'pcsk_6vppJn_Lad4mjWDcgb2rgq12WaePq71nrigPZYCk41TRzKP3D5W4qqJkbWWCm9ZR2iAUCY'

In [31]:
from pinecone import ServerlessSpec

index_name = "ppt-rag"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
else:
    print("Index already exists")

index = pc.Index(index_name)


In [28]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [32]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(doc))]

vector_store.add_documents(documents=doc, ids=uuids)

['42915a9b-8144-4321-ad8b-b5b9483a11e0',
 '4167d2fa-f847-4b9e-be13-4c77359888ab',
 '50b9b2cb-4dcd-44b0-ac18-8ce75993aa48',
 'b6c86fa6-e50c-47af-9140-052b145701b8',
 '73a4ff3e-cb11-4235-be50-f51840169da6',
 '8695be97-cda8-4822-b303-64c0e335c617',
 '8107b72c-dce6-434b-a623-2411c5bb7f72',
 'e16ce669-bbf7-4a5f-ba37-a6f861f54301',
 'ca66ea63-f43e-4aff-99fd-d6bf9089df36',
 '121a4bc5-cfdc-4131-b944-961ab98de95c']

In [34]:
results = vector_store.similarity_search_with_score(" Conclusion Diabetes in Pakistan")
results

[(Document(id='42915a9b-8144-4321-ad8b-b5b9483a11e0', metadata={'slide_number': 1.0}, page_content='1: Diabetes in Pakistan An Overview of Challenges, Trends, and Solutions\n'),
  0.96282047),
 (Document(id='4167d2fa-f847-4b9e-be13-4c77359888ab', metadata={'slide_number': 2.0}, page_content='2: Introduction Diabetes is a growing health concern in Pakistan, affecting millions of people across all age groups.\n'),
  0.927557409),
 (Document(id='121a4bc5-cfdc-4131-b944-961ab98de95c', metadata={'slide_number': 10.0}, page_content='10: Conclusion Diabetes in Pakistan is a major public health challenge that requires collective efforts from government, NGOs, and individuals.'),
  0.909392118),
 (Document(id='50b9b2cb-4dcd-44b0-ac18-8ce75993aa48', metadata={'slide_number': 3.0}, page_content='3: Prevalence Pakistan ranks among the top 10 countries for highest diabetes prevalence, with over 33 million people affected.\n'),
  0.890386105)]

In [35]:
retriever = vector_store.as_retriever(search_kwargs={"k":2})
response = retriever.get_relevant_documents("prevelance")
response

/tmp/ipython-input-1873522634.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = retriever.get_relevant_documents("prevelance")


[Document(id='42915a9b-8144-4321-ad8b-b5b9483a11e0', metadata={'slide_number': 1.0}, page_content='1: Diabetes in Pakistan An Overview of Challenges, Trends, and Solutions\n'),
 Document(id='4167d2fa-f847-4b9e-be13-4c77359888ab', metadata={'slide_number': 2.0}, page_content='2: Introduction Diabetes is a growing health concern in Pakistan, affecting millions of people across all age groups.\n')]

In [36]:
result = vector_store.similarity_search("top 10",k=4)
result

[Document(id='42915a9b-8144-4321-ad8b-b5b9483a11e0', metadata={'slide_number': 1.0}, page_content='1: Diabetes in Pakistan An Overview of Challenges, Trends, and Solutions\n'),
 Document(id='4167d2fa-f847-4b9e-be13-4c77359888ab', metadata={'slide_number': 2.0}, page_content='2: Introduction Diabetes is a growing health concern in Pakistan, affecting millions of people across all age groups.\n'),
 Document(id='50b9b2cb-4dcd-44b0-ac18-8ce75993aa48', metadata={'slide_number': 3.0}, page_content='3: Prevalence Pakistan ranks among the top 10 countries for highest diabetes prevalence, with over 33 million people affected.\n'),
 Document(id='e16ce669-bbf7-4a5f-ba37-a6f861f54301', metadata={'slide_number': 8.0}, page_content='8: Current Challenges Limited awareness, poor access to healthcare, shortage of trained professionals, and cultural barriers.\n')]

#Chaining

In [37]:
!pip install -qU google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [55]:
from langchain_core.runnables import RunnableLambda
retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.invoke("which country is in top 10")

[Document(id='50b9b2cb-4dcd-44b0-ac18-8ce75993aa48', metadata={'slide_number': 3.0}, page_content='3: Prevalence Pakistan ranks among the top 10 countries for highest diabetes prevalence, with over 33 million people affected.\n')]

In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the following question using the provided context and give little bit info about it using LLM.

Question: {question}

Context: {context}

"""


In [75]:
prompt = ChatPromptTemplate.from_messages([("human", message)])

In [77]:

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
response = rag_chain.invoke("what are the types of diabetes")
print(response.text())

Based on the provided text, there are three main types of diabetes:

* **Type 1 Diabetes:** This type is insulin-dependent, meaning the body doesn't produce insulin.  This is an autoimmune disease where the body's immune system attacks the insulin-producing cells in the pancreas.

* **Type 2 Diabetes:** This type is often associated with lifestyle factors and diet.  It's characterized by insulin resistance, where the body's cells don't respond properly to insulin, and eventually, the pancreas may not produce enough insulin.

* **Gestational Diabetes:** This type develops during pregnancy and usually resolves after delivery.  Hormonal changes during pregnancy can impact how the body processes glucose.


**LLM Information:**  Large Language Models (LLMs) like the one I am using are trained on massive datasets of text and code.  This allows them to identify patterns and relationships in information,  like extracting the different types of diabetes from the provided text.  While I can prov